# Extra 

In [ ]:
  import warnings
  warnings.filterwarnings('ignore')

  import numpy as np
  import pandas as pd


  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import MinMaxScaler
  from sklearn.preprocessing import RobustScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/drive/My Drive/day1.csv'
day1 = pd.read_csv(my_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# extract X columns we want to use by name using .loc
day1 = day1.loc[:,['workingday','temp','atemp','hum', 'windspeed','tot']]
# and use iloc to extract columns by number
day1x = day1.iloc[:,0:5] 

In [ ]:
# X = training set, newx = test set
def kNN(X,Y,newx,k,regress=True,allK=False,leave1out=False,scaleX=True,scaler=StandardScaler()):

  import warnings
  warnings.filterwarnings('ignore')

  import numpy as np

  from sklearn.neighbors import KNeighborsRegressor
  from sklearn.neighbors import KNeighborsClassifier 
  from sklearn.neighbors import KNeighborsRegressor

  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import MinMaxScaler
  from sklearn.preprocessing import RobustScaler

  from statistics import mean 
  from statistics import mode
  from collections import Counter

  def kNNtype(neighbs,regress):
    if regress:
      knn = KNeighborsRegressor(n_neighbors=neighbs)
    else:
      knn = KNeighborsClassifier(n_neighbors=neighbs)
    return knn

  if scaler != StandardScaler():
    scaler=scaler

  if scaleX == True:
    # scale feature columns for test and train data 
    columns_X = [list(X.columns)]
    for feature in columns_X:
      X[feature] = scaler.fit_transform(X[feature])
      newx[feature] = scaler.fit_transform(newx[feature])

  knn_all = pd.DataFrame()
  if allK == True:
    if leave1out == True:
      nn_all = []
      for j in list(newx.index.values.tolist()):
      #for j in list(Y.index.values.tolist()):
        knn_row = []
        knn = kNNtype(k+1,regress)
        knn.fit(X, Y)
        test = pd.DataFrame(newx.loc[j,:])
        nn = knn.kneighbors(test.T)[1][0]
        for i in range(2,k+1):
          nn1 = nn[1:i] # leave one out
          test = list(Y.iloc[nn1])
          if regress:
            test = mean(test)
          else:
            c = Counter(test)
            l = list(c.values())
            ind = l.index(max(c.values()))
            test = list(c.keys())[ind]
            # count number of times the max class occurs and if there is a ty
            # choose the second class with the max if index is even
            if (l.count(max(l))) > 1 and (j % 2 !=0):
              l[ind] = 0
              ind = l.index(max(c.values()))
              test = list(c.keys())[ind2]

          knn_row.append(test)
        knn_row = pd.DataFrame(knn_row)
        knn_all = [knn_all, knn_row]
        knn_all = pd.concat(knn_all,axis=1, ignore_index=True)
        nn_all.append(list(nn1))
      nn_all = np.array(nn_all)
    else:
        for i in range(1,k+1):
          knn = kNNtype(i,regress)
          knn.fit(X, Y)
          test = knn.predict(newx)
          knn_row = pd.DataFrame(test).T
          knn_all = [knn_all, knn_row]
          knn_all = pd.concat(knn_all,axis=0, ignore_index=True)
        nn_all = knn.kneighbors(newx)[1]
  else:
    if leave1out == True:
      knn_row = []
      #for j in list(Y.index.values.tolist()):
      for j in list(newx.index.values.tolist()):
        knn = kNNtype(k,regress)
        knn.fit(X, Y)
        test = pd.DataFrame(newx.loc[j,:])
        nn = knn.kneighbors(test.T)[1][0]

        if (k % 2 != 0) and (regress == False):
           nn1 = nn[2:len(nn)]
        else:
            nn1 = nn[1:len(nn)]

        test = list(Y.iloc[nn1])
        if regress:
            test = mean(test)
        else:
          c = Counter(test)
          l = list(c.values())
          ind = l.index(max(c.values()))
          test = list(c.keys())[ind]
          # count number of times the max class occurs and if there is a ty
          # choose the second class with the max if index is even
          if (l.count(max(l))) > 1 and (j % 2 !=0):
            l[ind] = 0
            ind = l.index(max(c.values()))
            test = list(c.keys())[ind2]

        knn_row.append(test)
      knn_all = pd.DataFrame(knn_row).T
      nn_all = nn1
    else:
        knn = kNNtype(k,regress)
        knn.fit(X, Y)
        test = knn.predict(newx)
        knn_all = pd.DataFrame(test)
        nn_all = knn.kneighbors(newx)[1]

  return knn_all, nn_all

In [ ]:
def my_mode(list):
  from collections import Counter 
  Ks_and_counts = dict(Counter(list))
  max_count = max(Ks_and_counts.values())
  most_common_Ks = []
  for k, count in Ks_and_counts.items():
    if count == max_count: 
      most_common_Ks.append(k)
  if len(most_common_Ks) == len(list):
    return 
  else:
    return most_common_Ks[0]


In [ ]:
def poulation_variance(list):
  from statistics import mean
  list_mean = mean(list)
  sum_difs = 0
  for item in list:
    diff_squared = (item - list_mean)**2 
    sum_difs += diff_squared 
  variance = sum_difs/len(list)
  return variance

def bestK(X,Y,maxK, start, stop):
  # X = feature predictors, Y = value to be predicted, maxK = maximum k value to test, maxRow = maximum row to check the best k
  
  import statistics 
  import numpy as np

  allpreds, nn = kNN(X,Y,X,maxK,allK = True,leave1out = True) 
  list_of_best_ks = []
  for i in range(start, stop):
    value = Y[i]
    differences = []
    for j in range(len(Y)):
      for prediction in allpreds[j]:
        difference = abs(value - prediction)
        differences.append(difference)
    indicies = np.argsort(differences)
    best_k = indicies[0] + 2
    list_of_best_ks.append(best_k)
  common_k = my_mode(list_of_best_ks)  ### Note: I altered my_mode function from question 1; seen above
  return common_k

def p_hack(X,Y,n_subsets,maxK):
  # X = feature predictors, Y = value to be predicted, maxK = maximum K to test, n_subsets = number of subsets from data
  import statistics

  subset_length = int(round(len(X)/n_subsets))
  list_of_best_Ks = []
  for i in range(n_subsets):
    start = i*subset_length
    stop = (i*subset_length) + subset_length
    common_k = bestK(X[start:stop],Y[start:stop],maxK,start, stop) ### Note: I generated a new bestK function for the p_hack function seen above
    list_of_best_Ks.append(common_k)
  new_list = list(filter(None, list_of_best_Ks)) 
  var_in_k_value_between_subset = poulation_variance(new_list)  ### Note: my own population variance function is seen above 
  return round(var_in_k_value_between_subset, 4)


  

In [ ]:
X = day1.iloc[:,0:5] 
Y = day1.tot
maxK = 4   # maxK cannot be larger than len(X)/n_subsets
n_subsets = 70    # in this case, each subset will be of length 10


p_hack = p_hack(X,Y,n_subsets,maxK)
print(p_hack)


69.9623
